In [26]:
import pandas as pd
import re
import openpyxl

# Load datasets
weather_data = pd.read_csv('Weather_data.csv')
migration_data = pd.read_excel('net_migration_by_country.xlsx', engine='openpyxl')
national_data = pd.read_csv('Subnational_unit.csv')





In [27]:
print(national_data)

    ISO3 National Unit      Name Admin Level          Metric  \
0    COL      Colombia  Amazonas      admin1  Incidence Rate   
1    COL      Colombia  Amazonas      admin1  Incidence Rate   
2    COL      Colombia  Amazonas      admin1  Incidence Rate   
3    COL      Colombia  Amazonas      admin1  Incidence Rate   
4    COL      Colombia  Amazonas      admin1  Incidence Rate   
..   ...           ...       ...         ...             ...   
820  BRA        Brazil  Amazonas      admin1  Mortality Rate   
821  BRA        Brazil  Amazonas      admin1  Mortality Rate   
822  BRA        Brazil  Amazonas      admin1  Mortality Rate   
823  BRA        Brazil  Amazonas      admin1  Mortality Rate   
824  BRA        Brazil  Amazonas      admin1  Mortality Rate   

                       Units  Year      Value  
0         Cases per Thousand  2000  22.550805  
1         Cases per Thousand  2001  35.036803  
2         Cases per Thousand  2002  30.800287  
3         Cases per Thousand  2003  27.

In [ ]:
weather_data.head() 

,Country,City,month,Avg. Temperature °C (°F),Min. Temperature °C (°F),Max. Temperature °C (°F),Precipitation / Rainfall mm (in),Humidity(%),Rainy days (d),avg. Sun hours (hours)
0,Columbia,Valle del Cauca,January,20.3 °C\n(68.5) °F,17.2 °C\n(63) °F,23.9 °C\n(74.9) °F,307\n(12),83%,19,7.6
1,Columbia,Valle del Cauca,February,20.5 °C\n(68.8) °F,17.5 °C\n(63.5) °F,24.1 °C\n(75.4) °F,278\n(10),83%,17,7.8
2,Columbia,Valle del Cauca,March,20.7 °C\n(69.2) °F,17.7 °C\n(63.9) °F,24.3 °C\n(75.8) °F,308\n(12),82%,18,8.4
3,Columbia,Valle del Cauca,April,20.8 °C\n(69.5) °F,18.1 °C\n(64.5) °F,24.3 °C\n(75.7) °F,320\n(12),82%,17,9.0
4,Columbia,Valle del Cauca,May,20.8 °C\n(69.5) °F,17.8 °C\n(64.1) °F,24.3 °C\n(75.7) °F,221\n(8),82%,15,9.4


In [24]:
migration_data.head() 

,country_code,country_name,year,value
0,ABW,Aruba,1960,0
1,ABW,Aruba,1961,-569
2,ABW,Aruba,1962,-609
3,ABW,Aruba,1963,-646
4,ABW,Aruba,1964,-684


In [39]:
# Normalize country names mapping
iso3_keep = {'BRA':'brazil', 'COL':'colombia', 'VEN':'venezuela'}

# Filter migration data for the three countries only
migration_filtered = migration_data[migration_data['country_code'].str.upper().isin(iso3_keep.keys())]
migration_filtered['Country'] = migration_filtered['country_code'].str.upper().map(iso3_keep)

# Aggregate migration by country and year, rename cols for merge
migration_agg = migration_filtered.groupby(['Country', 'year'])['value'].mean().reset_index()
migration_agg.rename(columns={'year': 'Year', 'value': 'MigrationNet'}, inplace=True)

# Normalize 'National Unit' in national data, lower case for matching
national_data['National Unit'] = national_data['National Unit'].str.lower()

# Filter national data for incidence rate metric only and aggregate by National Unit and Year
incidence_data = national_data[national_data['Metric'].str.lower() == 'incidence rate']
incidence_agg = incidence_data.groupby(['National Unit', 'Year'])['Value'].mean().reset_index()

# Merge incidence data with migration data on country and year
final_df = pd.merge(incidence_agg, migration_agg, how='left',
                    left_on=['National Unit', 'Year'], right_on=['Country', 'Year'])

# Drop redundant 'Country' column after join
final_df.drop(columns=['Country'], inplace=True)

# Show sample of final dataset
final_df

/var/folders/hg/gkyx41xn2gx8rv31p0sg09w40000gn/T/ipykernel_30308/2144803663.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  migration_filtered['Country'] = migration_filtered['country_code'].str.upper().map(iso3_keep)


,National Unit,Year,Value,MigrationNet
0,brazil,2000,23.612418,-1761.0
1,brazil,2001,14.573199,3453.0
2,brazil,2002,22.638925,6018.0
3,brazil,2003,15.110918,-16908.0
4,brazil,2004,21.312302,-5390.0
...,...,...,...,...
120,venezuela,2020,85.747905,-525142.0
121,venezuela,2021,67.273365,-525116.0
122,venezuela,2022,60.768447,NaN
123,venezuela,2023,65.279838,NaN


In [40]:
# Replace final_df with your dataframe variable name if needed
final_df.to_csv('mig_mal.csv', index=False)
